In [3]:
import requests

def encode(sentences: list[str]) -> list[list[float]]:
    res = requests.post("http://localhost:8080/encode", json={"sentences": sentences})

    if res.status_code != 200:
        res.raise_for_status()

    res_body = res.json()
    embeddings = res_body["embeddings"]

    if len(sentences) != len(embeddings):
        raise Exception("sentencesとEmbeddingAPIのレスポンスの数が一致しません。")

    return embeddings

In [20]:
from qdrant_client import QdrantClient
collection_name = "amazon_review"
client = QdrantClient(host="localhost", port=6333)


query = "なんかいい感じでした"
query_vector = encode([query])[0]
client.search(collection_name=collection_name, query_vector=query_vector)

[ScoredPoint(id=32, version=1, score=0.4247976, payload={'label': '0', 'text': 'すずらん\n\nまだ 届かず どうなってんのか❔Amazonで たのまなかったら よかった'}, vector=None),
 ScoredPoint(id=89, version=1, score=0.4237734, payload={'label': '0', 'text': '体は少し嫌な感覚が感じました\n\nニカッ月に使用し続けていたが、体は少し嫌な感覚が感じましたが、とても悪い体験があった。'}, vector=None),
 ScoredPoint(id=44, version=1, score=0.42211622, payload={'label': '0', 'text': 'こんな水臭い20世紀はじめて！\n\n先にレビューを見ればよかった！ こんなにも水臭い20世紀梨はじめて食べた、レビューにあったように、二十世紀でこんなにもまずいのがあるとは思わなかった。 アマゾンもベストセラーとかかくなよしかも一番目に、そういえば、先日も花咲蟹で一番目に出ていたのを買ったのも、あんなまずい、甘みのない花咲蟹もはじめて食べた、マヨネーズに砂糖を入れ食べないとにがみがあって。アマゾンで当分食べ物はやめとく。'}, vector=None),
 ScoredPoint(id=25, version=1, score=0.4028293, payload={'label': '0', 'text': 'がっかりです、、、\n\nレビューを読み、覚悟で購入しました。が、やはり匂いが変わってしまっていました。 以前に使っていた時はとても優しい香りで気に入っていたのですが、こうもきつい匂いに変わってしまって残念でなりません。 苦手なダウニーを思い出させる匂いです。どうか以前の香りに戻して欲しいです。 以前は香りが薄っすらでしたので規定量より多めに使っていましたが、今のは規定量の半分でもきついです。 残念ですが、破棄することになりそうです。'}, vector=None),
 ScoredPoint(id=9, version=1, score=0.38302654, payload={'label': '0'

In [30]:
from qdrant_client.models import Filter, FieldCondition, MatchValue


client.search(
    collection_name=collection_name, 
    query_vector=query_vector, 
    query_filter=Filter(
        must=[
            FieldCondition(key="label", match=MatchValue(value="1"))
        ]
    ),
    limit = 3
)

[ScoredPoint(id=1747, version=18, score=0.6678374, payload={'label': '1', 'text': '本物らしく見える\n\nかわいいかったです。また付けてはいませんが、おもったよりよかった'}, vector=None),
 ScoredPoint(id=1778, version=18, score=0.605243, payload={'label': '1', 'text': 'いまいちでした。\n\nうっかりクリックしましたが、味が他といまいちってかんじですね。'}, vector=None),
 ScoredPoint(id=1600, version=16, score=0.58639336, payload={'label': '1', 'text': 'ギラギラ\n\n見た目良さそうなので買いましたがギラギラしすぎて 親父に向いていないかも？'}, vector=None)]

In [19]:
from qdrant_client.models import Filter, FieldCondition, MatchValue, MatchText

client.search(
    collection_name=collection_name, 
    query_vector=query_vector, 
        query_filter=Filter(
        must=[
            FieldCondition(key="text", match=MatchText(text="砂糖"))
        ]
    ),
    limit = 3
)


[]